In [1]:
# pip install spacy

In [6]:
# pip install regex

In [41]:
# pip install scikit-learn

In [42]:
# pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from concurrent.futures import ProcessPoolExecutor
from fuzzywuzzy import fuzz
from sklearn.neighbors import NearestNeighbors
import spacy
from spacy import displacy
import string
import regex as re

/var/folders/pc/skjhh69x6c365spmwqcs6p9h0000gn/T/ipykernel_60073/1387260288.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
nlp = spacy.load("en_core_web_sm")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
cleaned_capes = pd.read_csv('cleaned_capes_data.csv')

In [4]:
cleaned_capes

,Unnamed: 0,Instructor,Department,Course,Quarter,Year,Percentage_Responded,Total_Enrolled,Total_Response,Percentage_Recommended_Class,Percentage_Recommended_Professor,Study_Hours_per_Week,Average_Grade_Received,Average_Grade_Received_missing
0,0,Butler Elizabeth Annette,AAS,AAS 10,SP,2023,72.73,66,48,93.5,100.0,2.80,3.67,False
1,1,Butler Elizabeth Annette,AAS,AAS 170,SP,2023,35.00,20,7,100.0,100.0,2.50,3.92,False
2,2,Jones Ian William Nasser,ANAR,ANAR 111,SP,2023,18.75,16,3,100.0,100.0,3.83,3.46,True
3,3,Shtienberg Gilad,ANAR,ANAR 115,SP,2023,23.08,26,6,100.0,83.3,3.83,3.07,False
4,4,Braswell Geoffrey E.,ANAR,ANAR 155,SP,2023,40.91,22,9,100.0,100.0,5.17,4.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62214,63358,Li Huai,VIS,VIS 105D,SU,2007,76.47,17,13,100.0,100.0,2.33,3.79,True
62215,63359,Guerrero Raul M.,VIS,VIS 106A,SU,2007,87.50,16,14,92.9,92.9,7.21,3.58,True
62216,63360,Mangolte Babette,VIS,VIS 194S,SU,2007,71.25,80,57,74.1,26.4,4.27,3.00,True
62217,63361,Holland Nicole Murphy,VIS,VIS 22,SU,2007,82.50,40,33,100.0,96.7,4.32,3.41,True


In [5]:
cleaned_capes = cleaned_capes[cleaned_capes['Department'] != "ANPR"]
cleaned_capes

,Unnamed: 0,Instructor,Department,Course,Quarter,Year,Percentage_Responded,Total_Enrolled,Total_Response,Percentage_Recommended_Class,Percentage_Recommended_Professor,Study_Hours_per_Week,Average_Grade_Received,Average_Grade_Received_missing
0,0,Butler Elizabeth Annette,AAS,AAS 10,SP,2023,72.73,66,48,93.5,100.0,2.80,3.67,False
1,1,Butler Elizabeth Annette,AAS,AAS 170,SP,2023,35.00,20,7,100.0,100.0,2.50,3.92,False
2,2,Jones Ian William Nasser,ANAR,ANAR 111,SP,2023,18.75,16,3,100.0,100.0,3.83,3.46,True
3,3,Shtienberg Gilad,ANAR,ANAR 115,SP,2023,23.08,26,6,100.0,83.3,3.83,3.07,False
4,4,Braswell Geoffrey E.,ANAR,ANAR 155,SP,2023,40.91,22,9,100.0,100.0,5.17,4.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62214,63358,Li Huai,VIS,VIS 105D,SU,2007,76.47,17,13,100.0,100.0,2.33,3.79,True
62215,63359,Guerrero Raul M.,VIS,VIS 106A,SU,2007,87.50,16,14,92.9,92.9,7.21,3.58,True
62216,63360,Mangolte Babette,VIS,VIS 194S,SU,2007,71.25,80,57,74.1,26.4,4.27,3.00,True
62217,63361,Holland Nicole Murphy,VIS,VIS 22,SU,2007,82.50,40,33,100.0,96.7,4.32,3.41,True


In [6]:
cleaned_income = pd.read_csv('fully_cleaned_income.csv')
income_df = cleaned_income
income_df

,Unnamed: 0,Year,Name,First Name,Last Name,Job Title,Income
0,12350,2010,ERNEST L III AARON,ERNEST L III,AARON,"TECHNICIAN, DEVELOPMENT, IV",65210.90
1,12351,2010,LOIS M. AARON,LOIS M.,AARON,TRANSLATOR/INTERP FOR THE DEAF,1891.54
2,12352,2010,GREGORY A. AARONS,GREGORY A.,AARONS,ASSOCIATE PROF IN RES-HCOMP,170724.59
3,12353,2010,CHARLES O ABAD,CHARLES O,ABAD,"TECHNICIAN,PATIENT DIALYSI, I",45102.64
4,12354,2010,CONCEPCION ABUTI ABAD,CONCEPCION ABUTI,ABAD,"CLERK, SR/SECRETARY",29320.17
...,...,...,...,...,...,...,...
362597,548579,2022,THOMAS ZUST,THOMAS,ZUST,ELECTR TCHN,68109.00
362598,548580,2022,MARGARET ZVIRZDIN,MARGARET,ZVIRZDIN,PAT RCDS ABSTRACTOR 3,94538.00
362599,548581,2022,JANE ZWERNEMAN,JANE,ZWERNEMAN,BLANK AST 3,90684.00
362600,548582,2022,MEGAN ZYNKIAN,MEGAN,ZYNKIAN,CLIN NURSE 3,117645.00


In [11]:
# Removes punctuation from a string input
def remove_punctuation(input_string):
    
    clean_punc = str(re.sub(r'[^a-zA-Z1-9\s]', '', input_string))
    
#     translator = str.maketrans("", "", string.punctuation)
#     translator = str(translator)
#     print(translator)
    
#     clean_string = input_string.translate(translator)
#     print("success")
    
    

    clean_string = clean_punc.lower()
    
    return clean_string

# First, we shrink the income data to include only professor 

In [8]:
unique_capes_professor = pd.DataFrame(pd.DataFrame(cleaned_capes['Instructor'].unique())[0].apply(remove_punctuation))
print(unique_capes_professor.iloc[2500:2550])

                                      0
2500                    haviland john b
2501               denham aaron renfrew
2502             ramirez belinda cherie
2503           alvarezcabanillas miguel
2504          villa rodriguez elizabeth
2505             lauberth shannon marie
2506                     bennett eric j
2507                     muotri alysson
2508                         lin chihho
2509                    burg suzanne mm
2510                      tsaur lihfeng
2511          freiling christopher fran
2512                   kriegman david j
2513                     yun kenneth yi
2514                machina mark joseph
2515                  williams ebonee p
2516                stokes malcolm dale
2517            rodriguez angelica rita
2518                       vaida florin
2519                        navarro ana
2520                       mills paul j
2521                      gibson carl h
2522                      pelayo alvaro
2523                        deng yanxia


In [9]:
# Create a new column "Name" by combining "First Name" and "Last Name"
income_df['Name'] = income_df['First Name'] + ' ' + income_df['Last Name']

name_index = 2

income_df.insert(name_index, 'Name', income_df.pop('Name'))
income_df

,Unnamed: 0,Year,Name,First Name,Last Name,Job Title,Income
0,12350,2010,ERNEST L III AARON,ERNEST L III,AARON,"TECHNICIAN, DEVELOPMENT, IV",65210.90
1,12351,2010,LOIS M. AARON,LOIS M.,AARON,TRANSLATOR/INTERP FOR THE DEAF,1891.54
2,12352,2010,GREGORY A. AARONS,GREGORY A.,AARONS,ASSOCIATE PROF IN RES-HCOMP,170724.59
3,12353,2010,CHARLES O ABAD,CHARLES O,ABAD,"TECHNICIAN,PATIENT DIALYSI, I",45102.64
4,12354,2010,CONCEPCION ABUTI ABAD,CONCEPCION ABUTI,ABAD,"CLERK, SR/SECRETARY",29320.17
...,...,...,...,...,...,...,...
362597,548579,2022,THOMAS ZUST,THOMAS,ZUST,ELECTR TCHN,68109.00
362598,548580,2022,MARGARET ZVIRZDIN,MARGARET,ZVIRZDIN,PAT RCDS ABSTRACTOR 3,94538.00
362599,548581,2022,JANE ZWERNEMAN,JANE,ZWERNEMAN,BLANK AST 3,90684.00
362600,548582,2022,MEGAN ZYNKIAN,MEGAN,ZYNKIAN,CLIN NURSE 3,117645.00


In [10]:
income_df = income_df.assign(Name = income_df["Name"].apply(remove_punctuation))
income_df


,Unnamed: 0,Year,Name,First Name,Last Name,Job Title,Income
0,12350,2010,ernest l iii aaron,ERNEST L III,AARON,"TECHNICIAN, DEVELOPMENT, IV",65210.90
1,12351,2010,lois m aaron,LOIS M.,AARON,TRANSLATOR/INTERP FOR THE DEAF,1891.54
2,12352,2010,gregory a aarons,GREGORY A.,AARONS,ASSOCIATE PROF IN RES-HCOMP,170724.59
3,12353,2010,charles o abad,CHARLES O,ABAD,"TECHNICIAN,PATIENT DIALYSI, I",45102.64
4,12354,2010,concepcion abuti abad,CONCEPCION ABUTI,ABAD,"CLERK, SR/SECRETARY",29320.17
...,...,...,...,...,...,...,...
362597,548579,2022,thomas zust,THOMAS,ZUST,ELECTR TCHN,68109.00
362598,548580,2022,margaret zvirzdin,MARGARET,ZVIRZDIN,PAT RCDS ABSTRACTOR 3,94538.00
362599,548581,2022,jane zwerneman,JANE,ZWERNEMAN,BLANK AST 3,90684.00
362600,548582,2022,megan zynkian,MEGAN,ZYNKIAN,CLIN NURSE 3,117645.00


In [28]:
def filter_income(name_1, name_2):
    doc1=nlp(name_1)
    doc2 = nlp(name_2)
    return doc1.similarity(doc2)

In [29]:
threshold = 0.85

income_df["Job Title"].value_counts().head(30)

LECT
PROF
TUT-NON STUDNT
LECTURER
MISCELLANEOUS
PROFESSOR
ACADEMIC COORD
COORD
RECALL

LECT
PROF
TUT-NON STUDNT
LECTURER
MISCELLANEOUS
PROFESSOR
ACADEMIC COORD
COORD
RECALL
POSTDOC-EMPLOYEE
ADMIN
ASSOC

In [18]:
allowed_strings = ['LECT', 'PROF', 'TUT-NON STUDNT', 
                   'LECTURER', 'MISCELLANEOUS', 'PROFESSOR', 
                   'ACADEMIC COORD', 'COORD', 'RECALL', 
                   'POSTDOC-EMPLOYEE', 'ADMIN', 'ASSOC']
income_df = income_df[income_df['Job Title'].fillna('').str.contains('|'.join(allowed_strings))]
income_df

,Unnamed: 0,Year,Name,First Name,Last Name,Job Title,Income
2,12352,2010,gregory a aarons,GREGORY A.,AARONS,ASSOCIATE PROF IN RES-HCOMP,170724.59
6,12356,2010,reynaldo j abad,REYNALDO J.,ABAD,ADMIN. SPECIALIST,46819.96
10,12360,2010,ruben abagyan,RUBEN,ABAGYAN,PROFESSOR-HCOMP,203054.99
13,12363,2010,henry don isaac abarbanel,HENRY DON ISAAC,ABARBANEL,PROFESSOR - ACADEMIC YEAR,178840.00
23,12373,2010,mark a abcede,MARK A,ABCEDE,STAFF RESEARCH ASSOC II,39169.00
...,...,...,...,...,...,...,...
362590,548572,2022,saichun zuo,SAICHUN,ZUO,POSTDOC-EMPLOYEE,52981.00
362591,548573,2022,xinxin zuo,XINXIN,ZUO,POSTDOC-EMPLOYEE,57224.00
362594,548576,2022,bruce zuraw,BRUCE,ZURAW,PROF-HCOMP,215001.00
362596,548578,2022,dmitri zurita,DMITRI,ZURITA,LECT-AY-1/9,25225.00


In [3]:
income_df.to_csv("only_profs_income.csv")

NameError: name 'income_df' is not defined

In [14]:
cleaned_capes = cleaned_capes[(cleaned_capes['Department'] != "ANPR") & (cleaned_capes['Year'] != 2023) & (cleaned_capes['Year'] >= 2010)] 

In [15]:
unique_capes_professor = pd.DataFrame(pd.DataFrame(cleaned_capes['Instructor'].unique())[0].apply(remove_punctuation))
# print(unique_capes_professor.iloc[2500:2550])

In [20]:
# print(unique_capes_professor.iloc[2500:2550])

In [28]:
unique_capes_professor[unique_capes_professor[0].str.contains("glynda")]

,0


In [21]:
# income_df[(income_df["Name"].str.contains("chakraborty"))&(income_df["Name"].str.contains("subrata"))]

In [3]:
# Removes punctuation from a string input
def remove_punctuation(input_string):
    
    clean_punc = str(re.sub(r'[^a-zA-Z1-9\s]', '', input_string))
    
#     translator = str.maketrans("", "", string.punctuation)
#     translator = str(translator)
#     print(translator)
    
#     clean_string = input_string.translate(translator)
#     print("success")
    
    

    clean_string = clean_punc.lower()
    
    return clean_string

In [4]:
def filter_income(name_1, name_2):
    doc1=nlp(name_1)
    doc2 = nlp(name_2)
    return doc1.similarity(doc2)

In [5]:
income_df = pd.read_csv("only_profs_income.csv")
# income_df

In [6]:
# Create a new column "Name" by combining "First Name" and "Last Name"
income_df['Name'] = income_df['First Name'] + ' ' + income_df['Last Name']

name_index = 2

income_df.insert(name_index, 'Name', income_df.pop('Name'))
# income_df

In [7]:
income_df = income_df.assign(Name = income_df["Name"].apply(remove_punctuation))
income_df

,Unnamed: 0.1,Unnamed: 0,Name,Year,First Name,Last Name,Job Title,Income
0,2,12352,gregory a aarons,2010,GREGORY A.,AARONS,ASSOCIATE PROF IN RES-HCOMP,170724.59
1,6,12356,reynaldo j abad,2010,REYNALDO J.,ABAD,ADMIN. SPECIALIST,46819.96
2,10,12360,ruben abagyan,2010,RUBEN,ABAGYAN,PROFESSOR-HCOMP,203054.99
3,13,12363,henry don isaac abarbanel,2010,HENRY DON ISAAC,ABARBANEL,PROFESSOR - ACADEMIC YEAR,178840.00
4,23,12373,mark a abcede,2010,MARK A,ABCEDE,STAFF RESEARCH ASSOC II,39169.00
...,...,...,...,...,...,...,...,...
89174,362590,548572,saichun zuo,2022,SAICHUN,ZUO,POSTDOC-EMPLOYEE,52981.00
89175,362591,548573,xinxin zuo,2022,XINXIN,ZUO,POSTDOC-EMPLOYEE,57224.00
89176,362594,548576,bruce zuraw,2022,BRUCE,ZURAW,PROF-HCOMP,215001.00
89177,362596,548578,dmitri zurita,2022,DMITRI,ZURITA,LECT-AY-1/9,25225.00


In [8]:
capes_df = pd.read_csv("cleaned_capes_data.csv")
# capes_df

In [9]:
capes_df = capes_df[(capes_df['Department'] != "ANPR") & (capes_df['Year'] != 2023) & (capes_df['Year'] >= 2010)] 

In [10]:
print(income_df.shape)
print(capes_df.shape)

(89179, 8)
(51550, 14)


In [11]:
unique_names = capes_df['Instructor'].unique()
unique_capes_df = pd.DataFrame({'Instructor': unique_names})
# unique_capes_df

In [12]:
unique_capes_df = pd.DataFrame(pd.DataFrame(capes_df['Instructor'].unique())[0].apply(remove_punctuation))
unique_capes_df

,0
0,butler elizabeth annette
1,fortier jana
2,smith neil gordon
3,jones ian william nasser
4,goldstein paul s
...,...
4865,weiss ray f
4866,barton thomas
4867,lindseth brian s
4868,deluca mara


In [13]:
instructor_dict = dict.fromkeys(unique_capes_df[0], 0)
# print(instructor_dict)

In [14]:
unique_instructor_names = income_df['Name'].unique()
unique_income_df = pd.DataFrame({'Instructor': unique_instructor_names})
unique_income_df

,Instructor
0,gregory a aarons
1,reynaldo j abad
2,ruben abagyan
3,henry don isaac abarbanel
4,mark a abcede
...,...
25969,maryia zosimava
25970,xun zou
25971,simone zuffa
25972,mark zumberge


In [15]:
print(unique_income_df.shape)
print(unique_capes_df.shape)

(25974, 1)
(4870, 1)


In [16]:
# Set a similarity threshold
threshold = 0.85

# Create vectors for each name using SpaCy embeddings
income_vectors = [nlp(name).vector for name in unique_income_df['Instructor']]
capes_vectors = [nlp(name).vector for name in unique_capes_df[0]]

# Use Nearest Neighbors to find potential matches efficiently
neigh = NearestNeighbors(metric='cosine')
neigh.fit(capes_vectors)

merged_data = []

# Find nearest neighbors for each income name
for i, income_vector in enumerate(income_vectors):
    distances, indices = neigh.kneighbors([income_vector])
    
    # Check if the nearest neighbor is within the threshold
    if distances[0][0] <= (1 - threshold):
        cape_name = unique_capes_df[0].iloc[indices[0][0]]
        merged_data.append({'Income_Name': unique_income_df['Instructor'].iloc[i], 'Capes_Name': cape_name, 'Similarity': 1 - distances[0][0]})

# Create the merged_df
merged_df = pd.DataFrame(merged_data)

In [18]:
# APPROACH ABOVE WORKS WITH TIME COMPLEXITY O(n logn) BUT IS INACCURATE
merged_df

,Income_Name,Capes_Name,Similarity
0,reynaldo j abad,cottrell garrison w,0.882427
1,henry don isaac abarbanel,abarbanel henry don isaac,0.957074
2,stephanie louise abel,angle kyle james,0.921564
3,mohammad mabrou aboulwafa,katz faith erin heffernan,0.896802
4,karen g abrams,kline matthew john,0.915660
...,...,...,...
15699,daniel zielinski,cole michael,0.887134
15700,maryia zosimava,johnson eric lynn,0.917130
15701,xun zou,liu zhaowei,0.854998
15702,mark zumberge,kempton mark condie,0.871357


In [ ]:
# Set a similarity threshold
threshold = 85  # Adjust the threshold as needed

# Initialize an empty list to store the results
merged_data = []

# Find matches using FuzzyWuzzy
for income_name in unique_income_df['Instructor']:
    matches = [cape_name for cape_name in unique_capes_df[0] if fuzz.token_set_ratio(income_name, cape_name) >= threshold]
    
    # Add the results to the merged_data
    for cape_name in matches:
        similarity = fuzz.token_set_ratio(income_name, cape_name)
        merged_data.append({'Income_Name': income_name, 'Capes_Name': cape_name, 'Similarity': similarity})

# Create the merged_df
merged_df = pd.DataFrame(merged_data)

In [61]:
# Set a similarity threshold
threshold = 85  # Adjust the threshold as needed

# Initialize an empty list to store the results
merged_data = []

def find_matches(income_name):
    matches = [cape_name for cape_name in unique_capes_df[0] if fuzz.token_set_ratio(income_name, cape_name) >= threshold]
    return [{'Income_Name': income_name, 'Capes_Name': cape_name, 'Similarity': fuzz.token_set_ratio(income_name, cape_name)} for cape_name in matches]

# Use parallel processing to find matches
with ProcessPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(find_matches, unique_income_df['Instructor']))

# Flatten the results list
merged_data = [result for sublist in results for result in sublist]

# Create the merged_df
merged_df = pd.DataFrame(merged_data)

Process SpawnProcess-11:
Process SpawnProcess-12:
Process SpawnProcess-10:
Process SpawnProcess-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
AttributeEr

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [39]:
merged_df

,Income_Name,Capes_Name,Similarity
0,reynaldo j abad,cottrell garrison w,0.882427
1,henry don isaac abarbanel,abarbanel henry don isaac,0.957074
2,stephanie louise abel,angle kyle james,0.921564
3,mohammad mabrou aboulwafa,katz faith erin heffernan,0.896802
4,karen g abrams,kline matthew john,0.915660
...,...,...,...
15699,daniel zielinski,cole michael,0.887134
15700,maryia zosimava,johnson eric lynn,0.917130
15701,xun zou,liu zhaowei,0.854998
15702,mark zumberge,kempton mark condie,0.871357


In [40]:
doc1 = nlp("reynaldo j abad")
doc2 = nlp("cottrell garrison w")
print(doc1.similarity(doc2))

0.8824272493104836


/var/folders/pc/skjhh69x6c365spmwqcs6p9h0000gn/T/ipykernel_59505/1234238622.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(doc1.similarity(doc2))


In [ ]:
merged_df = pd.DataFrame(columns=['Names', 'Similarity'])

for income_name in unique_income_df['Instructor']:
    # Filter names based on similarity threshold (0.85)
    similar_names = [cape_name for cape_name in unique_capes_df[0] if filter_income(income_name, cape_name) >= 0.85]

    # Add the results to the merged_df
    for similar_name in similar_names:
        merged_df = pd.concat([merged_df, pd.DataFrame({'Names': [income_name], 'Similarity': [filter_income(income_name, similar_name)]})], ignore_index=True)

/var/folders/pc/skjhh69x6c365spmwqcs6p9h0000gn/T/ipykernel_59108/1676999657.py:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/var/folders/pc/skjhh69x6c365spmwqcs6p9h0000gn/T/ipykernel_59108/3231182827.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([merged_df, pd.DataFrame({'Names': [income_name], 'Similarity': [filter_income(income_name, similar_name)]})], ignore_index=True)


In [ ]:
merged_df